
# Synchrotron Spectra With Cooling and SSA

In practice, synchrotron spectra are made much more complex than the
idealized scenarios often presented in textbooks (e.g. :footcite:t:`RybickiLightman` or
:footcite:t:`1970ranp.book.....P`). In the modern literature of supernova and GRB synchrotron
modeling, the effect of synchrotron self absorption (SSA) and radiative cooling of the electron
population are often included to better match observed spectra (e.g. :footcite:t:`GranotSari2002SpectralBreaks`).

.. hint::

    For a detailed discussion of the relevant theory and its implementation in Triceratops, see
    `synchrotron_theory`.

In Triceratops, we have implemented the general synchrotron SED including both SSA and cooling
effects in the class :class:`~radiation.synchrotron.SEDs.PowerLaw_Cooling_SSA_SynchrotronSED`.

In this example, we're going to simply demonstrate the shapes of the different spectra using
contrived values of the various break frequencies. In practice, models relying on these
spectra use them in series with physically-motivated values for the break frequencies and peak flux.
This example is primarily intended to illustrate the different spectral regimes that can arise
when both cooling and SSA are included.

## Setup

To get started, we'll need to import the
:class:`~radiation.synchrotron.SEDs.PowerLaw_Cooling_SSA_SynchrotronSED` class
from Triceratops, along with some standard packages for numerical work and plotting.

Synchrotron SEDs (all descended from :class:`~radiation.synchrotron.SEDs.SynchrotronSED`) need
to be instantiated before use (so that hyper-parameters and flags for various choices can be set). Once instantiated
we can define a frequency grid and some fixed phenomenological parameters for the spectra and then evaluate
the SED for different values of the relevant frequencies.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u

from triceratops.radiation.synchrotron.SEDs import PowerLaw_Cooling_SSA_SynchrotronSED
from triceratops.utils.plot_utils import set_plot_style

# Instantiate the SED.
sed = PowerLaw_Cooling_SSA_SynchrotronSED()

## Parameters
To get started, we'll want to define a grid of frequencies to evaluate the SED on,
along with some fixed phenomenological parameters for the spectra. We'll use a very
wide frequency grid to ensure we capture all relevant features of the spectra, including
the SSA low-frequency turnover and the high frequency cutoff.

Let's go ahead and set things up and get a basic spectrum generated.



In [ ]:
nu = np.geomspace(1e7, 1e20, 1000) * u.Hz

# Fix the phenomenological parameters
# nu_m: minimum injection frequency
# nu_c: cooling frequency
# nu_max: maximum frequency cutoff
# F_peak: peak flux density
nu_m = 1e7 * u.Hz  # Injection frequency
nu_max = 1e18 * u.Hz  # High-frequency cutoff
nu_c = 1e14 * u.Hz  # Cooling frequency
F_peak = 100 * u.mJy  # Peak flux density

# Hyper-parameters
# There are a couple parameters that control the shape of the spectrum
# in addition to the break frequencies and peak flux. These are:
# - p: power-law index of the electron distribution
# - s: smoothing parameter.
# - omega: (effective) solid angle of the emission region. This is often
#   calculated as A/D_A^2 where A is the area of the emitting region
#   and D_A is the angular diameter distance to the source.
# - gamma_m: minimum Lorentz factor of the electron distribution
p = 3.0
s = -0.01
omega = 0.5 * np.pi * (1e16 * u.cm) ** 2 / (1 * u.Mpc) ** 2
gamma_m = 1

# Compute and generate the plot.
flux_density = sed.sed(
    nu,
    nu_m,
    nu_c,
    F_peak,
    nu_max=nu_max,
    omega=omega,
    p=p,
    s=s,
    gamma_m=gamma_m,
)

# Create a plot.
set_plot_style()
plt.figure(figsize=(8, 6))
plt.loglog(nu, flux_density.to_value("mJy"), label="Cooling + SSA SED", color="C0")
plt.axvline(nu_m.value, color="C1", linestyle="--", label=r"$\nu_m$ (Injection Freq.)")
plt.axvline(nu_c.value, color="C2", linestyle="--", label=r"$\nu_c$ (Cooling Freq.)")
plt.axvline(nu_max.value, color="C3", linestyle="--", label=r"$\nu_{max}$ (Max Freq. Cutoff)")
plt.ylim([1e-10, 1e1])
plt.xlabel("Frequency (Hz)")
plt.ylabel("Flux Density (mJy)")
plt.title("Synchrotron SED with Cooling and SSA")
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.3)

plt.show()

We can also plot the log-log derivative so we can see the different spectral regimes more clearly.



In [ ]:
log_nu = np.log10(nu.to_value(u.Hz))
log_flux = np.log10(flux_density.to_value(u.mJy))
dlog_flux_dlog_nu = np.gradient(log_flux, log_nu)

set_plot_style()
plt.figure(figsize=(8, 6))
plt.semilogx(nu, dlog_flux_dlog_nu, label="Log-Log Derivative", color="C0")
plt.axvline(nu_m.value, color="C1", linestyle="--", label=r"$\nu_m$ (Injection Freq.)")
plt.axvline(nu_c.value, color="C2", linestyle="--", label=r"$\nu_c$ (Cooling Freq.)")
plt.axvline(nu_max.value, color="C3", linestyle="--", label=r"$\nu_{max}$ (Max Freq. Cutoff)")
plt.xlabel("Frequency (Hz)")
plt.ylabel(r"$\frac{d \log F_\nu}{d \log \nu}$")
plt.title("Log-Log Derivative of Synchrotron SED with Cooling and SSA")
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.3)
plt.ylim([-5, 5])

plt.show()

from matplotlib.cm import ScalarMappable

## The Regimes of the Spectrum

The shape of the synchrotron SED depends primarily on whether the electron
population is cooling efficiently. We therefore divide the possible spectra
into three broad classes:

1. Non-cooling ($\nu_c \gg \nu_{\rm max}$)
2. Fast-cooling ($\nu_c \ll \nu_m$)
3. Slow-cooling ($\nu_m \ll \nu_c \ll \nu_{\rm max}$)

Within each class, synchrotron self-absorption (SSA) can further modify the
low-frequency behavior of the spectrum, leading to several distinct spectral
shapes depending on the relative ordering of $\nu_a$, $\nu_m$, and $\nu_c$.

In the following sections, we examine each cooling regime in turn and
illustrate the corresponding spectra.

### I. Non-Cooling Spectra

We begin with the **non-cooling** regime, in which radiative losses are
negligible over the dynamical timescale of the system. In this case, the
injected power-law electron distribution remains intact.

The optically thin spectrum peaks at the injection frequency
$\nu_m$, with the familiar low-frequency slope
$F_\nu \propto \nu^{1/3}$ and high-frequency slope
$F_\nu \propto \nu^{-(p-1)/2}$.

Synchrotron self-absorption introduces an additional low-frequency break at
$\nu_a$, below which the spectrum becomes optically thick and transitions
to $F_\nu \propto \nu^2$ (or, in some regimes,
$F_\nu \propto \nu^{5/2}$).

We illustrate the possible non-cooling spectra by varying the peak flux
density $F_{\nu,\mathrm{pk}}$, which changes the optical depth and
therefore shifts the location of $\nu_a$.



In [ ]:
from matplotlib.colors import LogNorm

F_peaks = np.geomspace(1e-1, 1e12, 10) * u.mJy
nu_c_nc = 1e20 * u.Hz  # effectively no cooling

log_nu = np.log10(nu.to_value(u.Hz))

set_plot_style()
fig, (ax_top, ax_bot) = plt.subplots(
    2,
    1,
    figsize=(9, 8),
    sharex=True,
    gridspec_kw={"height_ratios": [3, 1], "hspace": 0.05},
)

# Colormap
cmap = plt.cm.viridis
norm = LogNorm(
    vmin=F_peaks.min().value,
    vmax=F_peaks.max().value,
)

for Fp in F_peaks:
    flux_i = sed.sed(
        nu,
        nu_m,
        nu_c_nc,
        Fp,
        nu_max=nu_max,
        omega=omega,
        p=p,
        s=s,
        gamma_m=gamma_m,
    )

    color = cmap(norm(Fp.value))

    # --- Top panel: SED ---
    ax_top.loglog(
        nu,
        flux_i.to_value(u.mJy),
        color=color,
        lw=2,
    )

    # --- Bottom panel: derivative ---
    log_flux_i = np.log10(flux_i.to_value(u.mJy))
    dlogF_dlognu = np.gradient(log_flux_i, log_nu)

    ax_bot.semilogx(
        nu,
        dlogF_dlognu,
        color=color,
        lw=1.5,
    )

# Reference lines
ax_top.axvline(nu_m.value, color="k", ls="--", alpha=0.4)
ax_bot.axvline(nu_m.value, color="k", ls="--", alpha=0.4)

# Labels
ax_top.set_ylabel("Flux Density (mJy)")
ax_top.set_title("Non-Cooling Synchrotron Spectra with SSA")

ax_bot.set_xlabel("Frequency (Hz)")
ax_bot.set_ylabel(r"$\mathrm{d}\log F_\nu / \mathrm{d}\log \nu$")

# Grid
for ax in (ax_top, ax_bot):
    ax.grid(True, which="both", ls="--", alpha=0.3)

ax_bot.set_ylim(-3, 3)

# Colorbar
sm = ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # safe for all matplotlib versions
cbar = fig.colorbar(sm, ax=[ax_top, ax_bot], pad=0.02)
cbar.set_label(r"$F_{\nu,\mathrm{pk}}\ \mathrm{[mJy]}$")


plt.show()

### II. Fast-Cooling Spectra

We now consider the **fast-cooling** regime, defined by

\begin{align}\nu_c \ll \nu_m.\end{align}

In this regime, electrons cool efficiently below the injection Lorentz
factor, modifying the optically thin synchrotron spectrum. Ignoring
synchrotron self-absorption, the resulting spectral slopes are:

- $F_\nu \propto \nu^{1/3}$ for $\nu < \nu_c$,
- $F_\nu \propto \nu^{-1/2}$ for $\nu_c < \nu < \nu_m$,
- $F_\nu \propto \nu^{-p/2}$ for $\nu > \nu_m$.

Synchrotron self-absorption introduces an additional low-frequency break
at $\nu_a$. In the following example, we focus on the commonly
encountered ordering

\begin{align}\nu_a < \nu_c < \nu_m,\end{align}

and illustrate the effect of SSA by varying the peak flux density
$F_{\nu,\mathrm{pk}}$, which shifts $\nu_a$ while leaving the
optically thin structure unchanged.



In [ ]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LogNorm

# Peak fluxes to vary (controls SSA depth)
F_peaks = np.geomspace(1e-1, 1e12, 10) * u.mJy

# Fast-cooling setup: ν_c ≪ ν_m
nu_c_fc = 1e9 * u.Hz  # cooling frequency well below ν_m

log_nu = np.log10(nu.to_value(u.Hz))

set_plot_style()
fig, (ax_top, ax_bot) = plt.subplots(
    2,
    1,
    figsize=(9, 8),
    sharex=True,
    gridspec_kw={"height_ratios": [3, 1], "hspace": 0.05},
)

# Colormap
cmap = plt.cm.viridis
norm = LogNorm(
    vmin=F_peaks.min().value,
    vmax=F_peaks.max().value,
)

for Fp in F_peaks:
    flux_i = sed.sed(
        nu,
        nu_m,
        nu_c_fc,
        Fp,
        nu_max=nu_max,
        omega=omega,
        p=p,
        s=s,
        gamma_m=gamma_m,
    )

    color = cmap(norm(Fp.value))

    # --- Top panel: SED ---
    ax_top.loglog(
        nu,
        flux_i.to_value(u.mJy),
        color=color,
        lw=2,
    )

    # --- Bottom panel: log-log derivative ---
    log_flux_i = np.log10(flux_i.to_value(u.mJy))
    dlogF_dlognu = np.gradient(log_flux_i, log_nu)

    ax_bot.semilogx(
        nu,
        dlogF_dlognu,
        color=color,
        lw=1.5,
    )

# Reference lines
ax_top.axvline(nu_c_fc.value, color="k", ls="--", alpha=0.4)
ax_top.axvline(nu_m.value, color="k", ls="--", alpha=0.4)

ax_bot.axvline(nu_c_fc.value, color="k", ls="--", alpha=0.4)
ax_bot.axvline(nu_m.value, color="k", ls="--", alpha=0.4)

# Labels
ax_top.set_ylabel("Flux Density (mJy)")
ax_top.set_title("Fast-Cooling Synchrotron Spectra with SSA")

ax_bot.set_xlabel("Frequency (Hz)")
ax_bot.set_ylabel(r"$\mathrm{d}\log F_\nu / \mathrm{d}\log \nu$")

# Grid
for ax in (ax_top, ax_bot):
    ax.grid(True, which="both", ls="--", alpha=0.3)

ax_bot.set_ylim(-4, 3)

# Colorbar
sm = ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cbar = fig.colorbar(sm, ax=[ax_top, ax_bot], pad=0.02)
cbar.set_label(r"$F_{\nu,\mathrm{pk}}\ \mathrm{[mJy]}$")


plt.show()

### III. Slow-Cooling Spectra

Finally, we consider the **slow-cooling** regime, defined by

\begin{align}\nu_m \ll \nu_c \ll \nu_{\rm max}.\end{align}

In this regime, radiative losses are inefficient for electrons near the
minimum injection Lorentz factor. As a result, the injected power-law
electron distribution remains intact below $\gamma_c$, and cooling
affects only the highest-energy electrons.

Ignoring synchrotron self-absorption, the optically thin synchrotron
spectrum in the slow-cooling regime consists of three power-law segments:

- $F_\nu \propto \nu^{1/3}$ for $\nu < \nu_m$,
- $F_\nu \propto \nu^{-(p-1)/2}$ for $\nu_m < \nu < \nu_c$,
- $F_\nu \propto \nu^{-p/2}$ for $\nu > \nu_c$.

Synchrotron self-absorption introduces an additional low-frequency break
at $\nu_a$, below which the spectrum becomes optically thick.
Depending on the ordering of $\nu_a$ relative to $\nu_m$ and
$\nu_c$, several distinct slow-cooling spectral shapes can arise.

In this example, we focus on the commonly encountered ordering

\begin{align}\nu_a < \nu_m < \nu_c,\end{align}

and illustrate the effect of SSA by varying the peak flux density
$F_{\nu,\mathrm{pk}}$, which shifts $\nu_a$ while leaving the
optically thin structure unchanged.



In [ ]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LogNorm

# Peak fluxes to vary (controls SSA depth)
F_peaks = np.geomspace(1e-1, 1e12, 10) * u.mJy

# Slow-cooling setup: ν_m ≪ ν_c ≪ ν_max
nu_c_sc = 1e16 * u.Hz  # cooling frequency well above ν_m

log_nu = np.log10(nu.to_value(u.Hz))

set_plot_style()
fig, (ax_top, ax_bot) = plt.subplots(
    2,
    1,
    figsize=(9, 8),
    sharex=True,
    gridspec_kw={"height_ratios": [3, 1], "hspace": 0.05},
)

# Colormap
cmap = plt.cm.viridis
norm = LogNorm(
    vmin=F_peaks.min().value,
    vmax=F_peaks.max().value,
)

for Fp in F_peaks:
    flux_i = sed.sed(
        nu,
        nu_m,
        nu_c_sc,
        Fp,
        nu_max=nu_max,
        omega=omega,
        p=p,
        s=s,
        gamma_m=gamma_m,
    )

    color = cmap(norm(Fp.value))

    # --- Top panel: SED ---
    ax_top.loglog(
        nu,
        flux_i.to_value(u.mJy),
        color=color,
        lw=2,
    )

    # --- Bottom panel: log-log derivative ---
    log_flux_i = np.log10(flux_i.to_value(u.mJy))
    dlogF_dlognu = np.gradient(log_flux_i, log_nu)

    ax_bot.semilogx(
        nu,
        dlogF_dlognu,
        color=color,
        lw=1.5,
    )

# Reference lines
ax_top.axvline(nu_m.value, color="k", ls="--", alpha=0.4)
ax_top.axvline(nu_c_sc.value, color="k", ls="--", alpha=0.4)

ax_bot.axvline(nu_m.value, color="k", ls="--", alpha=0.4)
ax_bot.axvline(nu_c_sc.value, color="k", ls="--", alpha=0.4)

# Labels
ax_top.set_ylabel("Flux Density (mJy)")
ax_top.set_title("Slow-Cooling Synchrotron Spectra with SSA")

ax_bot.set_xlabel("Frequency (Hz)")
ax_bot.set_ylabel(r"$\mathrm{d}\log F_\nu / \mathrm{d}\log \nu$")

# Grid
for ax in (ax_top, ax_bot):
    ax.grid(True, which="both", ls="--", alpha=0.3)

ax_bot.set_ylim(-4, 3)

# Colorbar
sm = ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cbar = fig.colorbar(sm, ax=[ax_top, ax_bot], pad=0.02)
cbar.set_label(r"$F_{\nu,\mathrm{pk}}\ \mathrm{[mJy]}$")


plt.show()